# Post-processing the reactive island data

Different parameter combination of $(\zeta, \lambda)$ 

In [13]:
#!/usr/bin/env python3
"""
Created on Thu 18 July 2019 11:19:25

@author: Shibabrat Naik
"""

import numpy as np
import scipy as sp
from scipy.optimize import fsolve

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
# mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-dark')
plt.style.use('seaborn') # use sans-serif fonts

# plt.style.use('default')

rcParams['figure.figsize'] = 5, 5

label_size = 25 #10, 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 25 #, 15

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0


import DeLeonBerne2dof
import importlib
importlib.reload(DeLeonBerne2dof)
import DeLeonBerne2dof as DB2dof



In [2]:

# Switch cases for different parameter values

# Fig. 3-A1
# ALPHA = 0.20;
# LAMBDA = 1.00;

# Fig. 3-A2
# ALPHA = 1.00;
# LAMBDA = 1.00;

# Fig. 3-B1
# ALPHA = 1.00;
# LAMBDA = 1.30;

# Fig. 3-B2
# ALPHA = 1.00;
# LAMBDA = 1.5;

# % Fig. 3-C1
# % ALPHA = 1.00;
# % LAMBDA = 2.00;

# % Fig. 3-C2
# ALPHA = 2.30;
# LAMBDA = 1.95;

def px_zero(x, *args):

    "function to solve minimum and maximum x on the SOS y = yw"

    Eval, EPSILON, Dx, ALPHA, LAMBDA = args[0:]
    
    H = lambda x: (Dx*(1 - np.exp(-LAMBDA*x))**2 - np.exp(-ALPHA*LAMBDA*x) + EPSILON)
    
    return (Eval - H(x))

def set_parameters(label):
    
    # ordering of the parameters: mass A, mass B, epsilon, Dx, alpha (z in paper), lambd
    massA = 8; massB = 8; EPSILON = 1.0; Dx = 10.0;
    
    if label == 'fig3A1/':
        ALPHA = 0.20;
        LAMBDA = 1.00;
    elif label == 'fig3A2/':
        ALPHA = 1.00;
        LAMBDA = 1.00;
    elif label == 'fig3B1/':
        ALPHA = 1.00;
        LAMBDA = 1.30;
    elif label == 'fig3B2/':
        ALPHA = 1.00;
        LAMBDA = 1.50;
    elif label == 'fig3C1/':
        ALPHA = 1.00;
        LAMBDA = 2.00;
    elif label == 'fig3C2/':
        ALPHA = 2.30;
        LAMBDA = 1.95;
    elif label == 'ZETA0.00-LAMBDA1.00/':
        ALPHA = 0.00;
        LAMBDA = 1.00;
    else:
        print('Incompatible arguments, check label string')
        
    params =  [massA, massB, EPSILON, Dx, ALPHA, LAMBDA]
    
    return params
        
        
def get_energybndry_intersect_sos(params, deltaEnergy, printflag):
    
    massA, massB, EPSILON, Dx, ALPHA, LAMBDA = params 
    saddleEnergy = EPSILON
    
    totalEnergy = saddleEnergy + deltaEnergy
    
    H = lambda x: (Dx*(1 - np.exp(-LAMBDA*x))**2 - np.exp(-ALPHA*LAMBDA*x) + EPSILON)
        
    # minumum and maximum of the energy boundary's x coordinate on the sos
    xMax = fsolve(px_zero,  0.5, args = (totalEnergy,EPSILON, Dx, ALPHA, LAMBDA))
    xMin = fsolve(px_zero,  -1, args = (totalEnergy,EPSILON, Dx, ALPHA, LAMBDA))
#     print(xMin, xMax)
    
    xGrid = np.linspace(xMin,xMax,1000)
    
    pxGridPos = np.zeros(np.size(xGrid))
    pxGridNeg = np.zeros(np.size(xGrid))
    
    for i in range(np.size(xGrid)):
        if (totalEnergy > H(xGrid[i])):
            pxGridPos[i] = sp.sqrt((2*massA)*(totalEnergy - H(xGrid[i])))
            
            pxGridNeg[i] = -sp.sqrt((2*massA)*(totalEnergy - H(xGrid[i])))
            
    
#     print('Excess energy:', deltaEnergy)    
    
    if printFlag:
        fig = plt.figure(figsize=(6, 6))
        ax = fig.gca()

    #     ax = axisH
    #     plt.rcParams['axes.labelweight'] = 'bold'
    #     plt.rcParams['ytick.major.size'] = 5    
    #     plt.rcParams['xtick.major.size'] = 5

    #    ax.xaxis.set_ticks([0, 5, 10, 15])
    #    ax.yaxis.set_ticks(np.linspace(-80,80,9,endpoint=True,dtype=int))

    #     ax.axis([0, rMax + 1, -80, 80])
    #     ax.set_xticklabels(ax.get_xticks(), weight='bold')
    #     ax.set_yticklabels(ax.get_yticks(), weight='bold')

        ax.plot(xGrid, pxGridPos, linewidth = 2, color = 'm')
        ax.plot(xGrid, pxGridNeg, linewidth = 2, color = 'm')

        ax.set_xlabel(r'$x$', fontsize = 25)
        ax.set_ylabel(r'$p_x$', fontsize = 25)
        plt.tick_params(axis = 'both', which = 'major', labelsize = 15)
        
        
    xGrid = np.append(xGrid, np.flip(xGrid,0))
    pxGrid = np.append(pxGridPos, np.flip(pxGridNeg,0))
    
#     print(np.shape(xGrid))
        
    return np.array([xGrid,pxGrid]).T



## Loading the manifold intersection data (warning: takes a bit of time)

In [3]:

# data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/data/DeLeon-Berne/' + \
#     'reaction-fraction/stable-mani-top-well/'
data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/system-bath/' + \
    'data-figures/DeLeon-Berne/reaction-fraction/stable-mani-top-well/'


# energy range discretization, this should match the data 
# deltaE_step = 0.01
# deltaE_start = 0.01;
# deltaE_stop = 0.51;
deltaE_step = 0.01
deltaE_start = 0.01;
deltaE_stop = 1.00;

deltaE_vals = np.arange(deltaE_start,deltaE_stop + deltaE_step,deltaE_step)

zeta_lambda_comb = ['ZETA0.00-LAMBDA1.00/','fig3A1/','fig3A2/','fig3B1/','fig3B2/','fig3C1/','fig3C2/']

tubemani_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))
energysurf_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))

printFlag = False
for k in range(len(zeta_lambda_comb)):
    for i in range(len(deltaE_vals)):
    
        if (float(deltaE_vals[i]) - int(deltaE_vals[i]) < 1e-12):    
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.0f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        else:
            try:
                data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                                  'xeU1_stable_branch1_eqPt1_DelE%.2f'%(deltaE_vals[i]) + \
                                  '_deleonberne.txt')
            except:
                data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                                  'xeU1_stable_branch1_eqPt1_DelE%.1f'%(deltaE_vals[i]) + \
                                  '_deleonberne.txt')
    

        tubemani_intersect_area[i,k] = DB2dof.get_area_closed_curve(data[:,(0,2)]) 

        # set parameters to calculate energy boundary on the section y = y_w
        params = set_parameters(zeta_lambda_comb[k])

        energy_boundary = get_energybndry_intersect_sos(params, 
                                                        deltaE_vals[i], printFlag)
        
        energysurf_intersect_area[i,k] = DB2dof.get_area_closed_curve(energy_boundary) 
        
        

        

## Plotting reaction fraction for small excess energy

In [16]:

%matplotlib
# print(tube_intersect_area)

ls_tick = 20
ls_axes = 25
mpl.rcParams['xtick.labelsize'] = ls_tick
mpl.rcParams['ytick.labelsize'] = ls_tick
mpl.rcParams['axes.labelsize'] = ls_axes


markertype = ['^-c','s-r','v-g','h-b','d-k','<-m','>-']
fig = plt.figure(figsize = (10,10))
ax = fig.gca()

for k in range(len(zeta_lambda_comb)):
#     k = 3
    params = set_parameters(zeta_lambda_comb[k])
    
#     ax.plot(deltaE_vals, tubemani_intersect_area[:,k], '-')
    line_handle = ax.plot(params[2] + deltaE_vals, \
                    tubemani_intersect_area[:,k]/energysurf_intersect_area[:,k], \
                    markertype[k], \
                    label = r'$\zeta = %.2f,\lambda = %.2f$'%(params[4],params[5]))
    
    ax.legend(fontsize = ls_tick)


ax.set_xlabel(r'Total energy, $E$', fontsize = ls_axes)
ax.set_ylabel(r'Reaction fraction, $T_{\rm react}(E)$', fontsize = ls_axes)

fig.savefig('temp.pdf', bbox_inches = 'tight')
plt.show()



Using matplotlib backend: MacOSX


## Visualizing the reactive islands

In [35]:


plt.style.use('default')
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'

plt.close('all')
params = set_parameters('fig3A2/')
# print(params)
deltaEnergy = 0.510
printFlag = True
energy_boundary = get_energybndry_intersect_sos(params, deltaEnergy, printFlag)

# energy_boundary = get_energybndry_intersect_sos(params,
#                                                 deltaE_vals[i], printFlag)

# fig = plt.figure(figsize = (8,8))
# ax = fig.gca()
# plt.plot(energy_boundary[:,0], energy_boundary[:,1],'-m')
# plt.plot(np.flip(energy_boundary[:,0],0), -np.flip(energy_boundary[:,1],0), '-k')
print(np.max(energy_boundary[:,0]),np.min(energy_boundary[:,0]))
print(np.max(energy_boundary[:,1]),np.min(energy_boundary[:,1]))



0.418232405019303 -0.36588592464709385
4.955804676874323 -4.955804676874323


## Loading the manifold intersection data (warning: takes a bit of time)

In [18]:


# data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/data/DeLeon-Berne/' + \
#     'reaction-fraction/stable-mani-top-well/'
data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/system-bath/' + \
    'data-figures/DeLeon-Berne/reaction-fraction/stable-mani-top-well/'

# energy range discretization, this should match the data
deltaE_start = 1.1;
deltaE_stop = 6.5;
deltaE_step = 0.1;

deltaE_vals = np.arange(deltaE_start,deltaE_stop + deltaE_step,deltaE_step)

zeta_lambda_comb = ['ZETA0.00-LAMBDA1.00/','fig3A1/','fig3B2/','fig3C2/']

tubemani_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))
energysurf_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))

printFlag = False
for k in range(len(zeta_lambda_comb)):
    for i in range(len(deltaE_vals)):
    
        if (float(deltaE_vals[i]) - int(deltaE_vals[i]) < 1e-12):    
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.0f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        else:
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.1f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        
        

        tubemani_intersect_area[i,k] = DB2dof.get_area_closed_curve(data[:,(0,2)]) 

        # set parameters to calculate energy boundary on the section y = y_w
        params = set_parameters(zeta_lambda_comb[k])

        energy_boundary = get_energybndry_intersect_sos(params, 
                                                        deltaE_vals[i], printFlag)
        
        energysurf_intersect_area[i,k] = DB2dof.get_area_closed_curve(energy_boundary) 
        


## Plotting reaction fraction for large excess energy

In [20]:

# print(tube_intersect_area)
%matplotlib

ls_tick = 20
ls_axes = 25
mpl.rcParams['xtick.labelsize'] = ls_tick
mpl.rcParams['ytick.labelsize'] = ls_tick
mpl.rcParams['axes.labelsize'] = ls_axes


markertype = ['^-c','s-r','d-k','>-']
fig = plt.figure(figsize = (10,10))
ax = fig.gca()

for k in range(len(zeta_lambda_comb)):
    params = set_parameters(zeta_lambda_comb[k])
    
#     ax.plot(deltaE_vals, tubemani_intersect_area[:,k], '-')
    line_handle = ax.plot(params[2] + deltaE_vals, \
                    tubemani_intersect_area[:,k]/energysurf_intersect_area[:,k], \
                    markertype[k], \
                    label = r'$\zeta = %.2f,\lambda = %.2f$'%(params[4],params[5]))
    
    ax.legend(fontsize = ls_tick)


ax.set_xlabel(r'Total energy, $E$', fontsize = ls_axes)
ax.set_ylabel(r'Reaction fraction, $T_{\rm react}(E)$', fontsize = ls_axes)

# fig.savefig('temp.pdf', bbox_inches = 'tight')
plt.show()



Using matplotlib backend: MacOSX
